# Recurrent Neural Networks

These are networks that have a different structure. Instead of passing the weights from one layer to the next, there is part of it that stays within the network, stored in a hidden state. 

This looks as follows:

<img src='./img/RNN-rolled.png', width=100px, height=100px >


or, to put it in a different way:

<img src='./img/RNN-unrolled.png'>



What can you do with recurrent neural networks? They are useful to keep longer term dependencies between input and output. 

This is kind of a problem very often in sequential decision making: the output makes sense after some complicated combination of different stimuli.

Many interesting use cases of RNN are described here:

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys

In [25]:
f = open('./data/female.txt','r')

In [26]:
text_lines = f.readlines()

In [27]:
text = ' '.join([t for t in text_lines])

In [16]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 56


In [17]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 10
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

nb sequences: 40566


In [18]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [19]:
# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [20]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [21]:
# train the model, output generated text after each iteration
for iteration in range(1, 5):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, epochs=2)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        print('\nGenerated')
        sys.stdout.write(generated)

        for i in range(20):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/2
40566/40566 [==============================] - 10s 239us/step - loss: 1.8394
Epoch 2/2
40566/40566 [==============================] - 9s 224us/step - loss: 1.0924

----- diversity: 0.2
----- Generating with seed: "nnaliese
 "

Generated
nnaliese
 Analia
 Analie
 Anal

----- diversity: 0.5
----- Generating with seed: "nnaliese
 "

Generated
nnaliese
 Analia
 Anana
 Anali

----- diversity: 1.0
----- Generating with seed: "nnaliese
 "

Generated
nnaliese
 Anala
 Andia
 Andodd

----- diversity: 1.2
----- Generating with seed: "nnaliese
 "

Generated
nnaliese
 Angobipea
 Aonifla
 

--------------------------------------------------
Iteration 2
Epoch 1/2
40566/40566 [==============================] - 9s 232us/step - loss: 0.9238
Epoch 2/2
40566/40566 [==============================] - 10s 240us/step - loss: 0.8158

----- diversity: 0.2
----- Generating with seed: "rmella
 Ca"

Generated
rmella
 Carmelle
 Carmeline
 M



KeyboardInterrupt: 